In [1]:
# This notebook is for pretraining and finetuning models... (GOOGLENET)

In [2]:
# Importing the necessary libraries #
import torch
import numpy as np
import torch.nn as nn
import torchvision.transforms.functional as F
import lightning as L
from typing import List
from lightning.pytorch import Trainer
from torchvision.datasets import MNIST
from torchvision import transforms
from torch.utils.data import DataLoader, random_split
from lightning.pytorch.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import precision_score
from torchmetrics import Accuracy
import utils
import os

In [3]:
torch.cuda.empty_cache()
torch.set_float32_matmul_precision("medium")

In [4]:
import wandb
from lightning.pytorch.loggers import WandbLogger
wandb.login()
# Initializing wandb logger #
wandb_logger = WandbLogger(
    entity="A2_DA6401_DL",
    project="GoogLeNet",       
)
wandb.login(key="5ef7c4bbfa350a2ffd3c198cb9289f544e3a0910")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: ae21b105 to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/joel/.netrc


True

In [ ]:
from torchvision import models
from torchvision.models import GoogLeNet_Weights

# Load pretrained GoogLeNet
model = models.googlenet(weights=GoogLeNet_Weights.IMAGENET1K_V1)

# Freeze all parameters
for param in model.parameters():
    param.requires_grad = False

model.fc = nn.Sequential(
    nn.Linear(1024, 512),
    nn.BatchNorm1d(512),
    nn.Mish(),
    nn.Dropout(0.3),
    nn.Linear(512, 10),
)

# Unfreeze the new layers for training
for param in model.fc.parameters():
    param.requires_grad = True

"""
# Unfreeze the new layers for training
for param in model.inception5a.parameters():
    param.requires_grad = True
for param in model.inception5b.parameters():
    param.requires_grad = True
"""

# Check model structure (optional)
print(model)


GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

In [6]:
# Normalization followed by google net
mean = [0.485, 0.456, 0.406]
std  = [0.229, 0.224, 0.225]

data_transforms = {
        "orient_" : transforms.Compose([
            utils.OrientReshape(size=(224,224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=mean, std=std)
        ]),
        "train_" : transforms.Compose([
            transforms.RandomHorizontalFlip(p = 0.3),
            transforms.RandomVerticalFlip(p = 0.3),
            transforms.RandomRotation(degrees=15),
            transforms.RandomAdjustSharpness(sharpness_factor=2, p=0.3),
            transforms.GaussianBlur(kernel_size=3),
            transforms.ToTensor(),
            transforms.RandomErasing(p = 0.2, scale=(0.02, 0.075)),
            transforms.Normalize(mean=mean, std=std)
        ])
    }


# Defining the dataloaders to be built
train_dataset, val_dataset, _ = utils.create_dataset_image_folder(path_=os.path.join(os.path.abspath(""), "nature_12K/inaturalist_12K/train/"), input_size=(224,224))
train_loader, val_loader = utils.create_dataloaders(batch_size=32, num_workers=2, train_dataset=train_dataset, val_dataset=val_dataset, is_data_aug=True, data_transforms = data_transforms)

# Setting up the callbacks to be used
early_stopping = EarlyStopping('val_acc', patience=5, mode="max")
checkpoint_callback = ModelCheckpoint(monitor="val_acc", dirpath="checkpoints/", filename="best-checkpoint_googlenet_nm", save_top_k=1, mode="max")
# Defining the model
lit_model = utils.Lightning_CNN(model = model)
# Training the Model
trainer = Trainer(max_epochs=50, precision="16-mixed", accelerator="auto", logger=wandb_logger, callbacks=[early_stopping, checkpoint_callback])
trainer.fit(lit_model, train_dataloaders=train_loader, val_dataloaders=val_loader)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


/home/joel/Pytorch_CUDA/virt_env/lib/python3.12/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: Checkpoint directory /home/joel/Pytorch_CUDA/checkpoints exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type               | Params | Mode 
----------------------------------------------------------
0 | model      | GoogLeNet          | 6.1 M  | train
1 | loss_fn    | CrossEntropyLoss   | 0      | train
2 | acc_metric | MulticlassAccuracy | 0      | train
----------------------------------------------------------
530 K     Trainable params
5.6 M     Non-trainable params
6.1 M     Total params
24.523    Total estimated model params size (MB)
231       Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [7]:
# Recalling the model weights from the callbacks
best_model_path = checkpoint_callback.best_model_path
lit_model = utils.Lightning_CNN.load_from_checkpoint(best_model_path, model=model)

# Getting the test dataloader
test_loader = utils.get_test_dataloader(os.path.join(os.path.abspath(""), "nature_12K/inaturalist_12K/val/"), data_transforms)
# Prediction of the test data
trainer = Trainer(logger=wandb_logger)
# Running prediction
predictions = trainer.test(model=lit_model, dataloaders=test_loader)

You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.7170000076293945     │
│         test_loss         │    0.8876169323921204     │
└───────────────────────────┴───────────────────────────┘